In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# Function to extract a random subset of data
def extract_subset(features, labels, subset_size=1000):
    idx = np.random.choice(features.shape[0], subset_size, replace=False)
    return features[idx], labels[idx]

# Function to initialize a decision tree node
def init_node(label=None):
    return {'label': label, 'left_child': None, 'right_child': None, 'split_feature': None, 'split_value': None}

# Function to check if a node is terminal
def is_terminal(node):
    return node['label'] is not None

# Function to categorize features
def categorize_features(features):
    feature_categories = []
    for i in range(features.shape[1]):
        distinct_values = np.unique(features[:, i])
        if isinstance(distinct_values[0], (int, float)) and len(distinct_values) > 10:
            feature_categories.append("continuous")
        else:
            feature_categories.append("discrete")
    return feature_categories

# Function to partition data based on a feature and threshold
def partition_data(features, split_feature, split_value):
    left_subset = np.where(features[:, split_feature] <= split_value)[0]
    right_subset = np.where(features[:, split_feature] > split_value)[0]
    return left_subset, right_subset

# Function to compute entropy
def compute_entropy(labels):
    label_freq = Counter(labels)
    total_count = len(labels)
    entropy = 0.0
    for freq in label_freq.values():
        prob = freq / total_count
        if prob > 0:
            entropy -= prob * np.log2(prob)
    return entropy

# Function to calculate information gain
def calc_info_gain(parent_labels, left_labels, right_labels):
    n = len(parent_labels)
    parent_entropy = compute_entropy(parent_labels)
    left_entropy = compute_entropy(left_labels)
    right_entropy = compute_entropy(right_labels)
    weighted_child_entropy = (len(left_labels) / n) * left_entropy + (len(right_labels) / n) * right_entropy
    return parent_entropy - weighted_child_entropy

# Function to determine the optimal split
def find_best_split(features, labels, feature_categories):
    max_gain = -np.inf
    optimal_feature = None
    optimal_value = None
    num_features = features.shape[1]

    for feature in range(num_features):
        if feature_categories[feature] == "continuous":
            potential_splits = np.unique(features[:, feature])
            for split in potential_splits:
                left_idx, right_idx = partition_data(features, feature, split)
                if len(left_idx) == 0 or len(right_idx) == 0:
                    continue
                gain = calc_info_gain(labels, labels[left_idx], labels[right_idx])
                if gain > max_gain:
                    max_gain = gain
                    optimal_feature = feature
                    optimal_value = split
        else:
            distinct_values = np.unique(features[:, feature])
            for value in distinct_values:
                left_idx, right_idx = partition_data(features, feature, value)
                if len(left_idx) == 0 or len(right_idx) == 0:
                    continue
                gain = calc_info_gain(labels, labels[left_idx], labels[right_idx])
                if gain > max_gain:
                    max_gain = gain
                    optimal_feature = feature
                    optimal_value = value

    return optimal_feature, optimal_value

# Function to construct a decision tree
def construct_tree(features, labels, feature_categories, max_features=None):
    if len(np.unique(labels)) == 1:
        return init_node(label=labels[0])

    if max_features is not None:
        feature_subset = np.random.choice(len(feature_categories), max_features, replace=False)
    else:
        feature_subset = range(len(feature_categories))

    max_gain = -np.inf
    optimal_feature = None
    optimal_value = None

    for feature in feature_subset:
        if feature_categories[feature] == "continuous":
            potential_splits = np.unique(features[:, feature])
            for split in potential_splits:
                left_idx, right_idx = partition_data(features, feature, split)
                if len(left_idx) == 0 or len(right_idx) == 0:
                    continue
                gain = calc_info_gain(labels, labels[left_idx], labels[right_idx])
                if gain > max_gain:
                    max_gain = gain
                    optimal_feature = feature
                    optimal_value = split
        else:
            distinct_values = np.unique(features[:, feature])
            for value in distinct_values:
                left_idx, right_idx = partition_data(features, feature, value)
                if len(left_idx) == 0 or len(right_idx) == 0:
                    continue
                gain = calc_info_gain(labels, labels[left_idx], labels[right_idx])
                if gain > max_gain:
                    max_gain = gain
                    optimal_feature = feature
                    optimal_value = value

    if optimal_feature is None:
        return init_node(label=Counter(labels).most_common(1)[0][0])

    left_idx, right_idx = partition_data(features, optimal_feature, optimal_value)

    if len(left_idx) == 0 or len(right_idx) == 0:
        return init_node(label=Counter(labels).most_common(1)[0][0])

    left_subtree = construct_tree(features[left_idx], labels[left_idx], feature_categories, max_features)
    right_subtree = construct_tree(features[right_idx], labels[right_idx], feature_categories, max_features)

    root = init_node()
    root['split_feature'] = optimal_feature
    root['split_value'] = optimal_value
    root['left_child'] = left_subtree
    root['right_child'] = right_subtree

    return root

# Function to make predictions using a decision tree
def tree_predict(features, tree):
    predictions = np.zeros(features.shape[0])
    for i, instance in enumerate(features):
        node = tree
        while not is_terminal(node):
            if instance[node['split_feature']] <= node['split_value']:
                node = node['left_child']
            else:
                node = node['right_child']
        predictions[i] = node['label']
    return predictions

# Main function to implement bagging and random forest with bias-variance analysis
def ensemble_learning(train_features, train_labels, test_features, test_labels, num_iterations=10, num_trees=20, subset_size=300):
    single_tree_predictions = np.zeros((num_iterations, test_features.shape[0]))
    bagged_predictions = np.zeros((num_iterations, test_features.shape[0]))

    feature_categories = categorize_features(train_features)

    for i in range(num_iterations):
        # Extract a subset of data
        features_subset, labels_subset = extract_subset(train_features, train_labels, subset_size)

        # Train a single decision tree
        single_tree = construct_tree(features_subset, labels_subset, feature_categories)
        single_tree_predictions[i] = tree_predict(test_features, single_tree)

        # Train an ensemble of decision trees
        ensemble_predictions = np.zeros((num_trees, test_features.shape[0]))
        for t in range(num_trees):
            features_bag, labels_bag = extract_subset(train_features, train_labels, subset_size)
            tree = construct_tree(features_bag, labels_bag, feature_categories)
            ensemble_predictions[t] = tree_predict(test_features, tree)

        # Aggregate predictions for bagged trees
        bagged_predictions[i] = np.mean(ensemble_predictions, axis=0)

    # Compute bias and variance
    single_tree_bias, single_tree_variance = compute_bias_variance(single_tree_predictions, test_labels)
    bagged_tree_bias, bagged_tree_variance = compute_bias_variance(bagged_predictions, test_labels)

    return single_tree_bias, single_tree_variance, bagged_tree_bias, bagged_tree_variance

# Main function to implement random forest with bias-variance analysis
def random_forest_learning(train_features, train_labels, test_features, test_labels, num_iterations=10, num_trees=20, subset_size=300, max_features=4):
    rf_single_tree_predictions = np.zeros((num_iterations, test_features.shape[0]))
    forest_predictions = np.zeros((num_iterations, test_features.shape[0]))

    feature_categories = categorize_features(train_features)

    for i in range(num_iterations):
        # Extract a subset of data
        features_subset, labels_subset = extract_subset(train_features, train_labels, subset_size)

        # Train a single random tree (using a subset of features)
        single_tree = construct_tree(features_subset, labels_subset, feature_categories, max_features)
        rf_single_tree_predictions[i] = tree_predict(test_features, single_tree)

        # Train a random forest
        forest_predictions_per_tree = np.zeros((num_trees, test_features.shape[0]))
        for t in range(num_trees):
            features_bag, labels_bag = extract_subset(train_features, train_labels, subset_size)
            tree = construct_tree(features_bag, labels_bag, feature_categories, max_features)
            forest_predictions_per_tree[t] = tree_predict(test_features, tree)

        # Aggregate predictions for the forest
        forest_predictions[i] = np.mean(forest_predictions_per_tree, axis=0)

    # Compute bias and variance
    rf_single_tree_bias, rf_single_tree_variance = compute_bias_variance(rf_single_tree_predictions, test_labels)
    forest_bias, forest_variance = compute_bias_variance(forest_predictions, test_labels)

    return rf_single_tree_bias, rf_single_tree_variance, forest_bias, forest_variance

# Function to compute bias and variance
def compute_bias_variance(predictions, true_labels):
    bias_squared = np.mean((np.mean(predictions, axis=0) - true_labels) ** 2)
    variance = np.mean(np.var(predictions, axis=0))
    return bias_squared, variance

def preprocess_dataset(dataframe):
    categorical_cols = dataframe.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        dataframe[col] = pd.Categorical(dataframe[col]).codes
    return dataframe

# Main execution function
def execute_analysis():
    # Load and preprocess the data
    column_names = ['age', 'job', 'marital', 'education', 'default', 'balance',
                    'housing', 'loan', 'contact', 'day', 'month', 'duration',
                    'campaign', 'pdays', 'previous', 'poutcome', 'label']

    train_dataset = pd.read_csv("train.csv", names=column_names)
    test_dataset = pd.read_csv("test.csv", names=column_names)

    # Preprocess the datasets
    train_dataset = preprocess_dataset(train_dataset)
    test_dataset = preprocess_dataset(test_dataset)

    # Separate features and labels
    train_features = train_dataset.drop('label', axis=1).values
    train_labels = np.where(train_dataset['label'] == 1, 1, -1)
    test_features = test_dataset.drop('label', axis=1).values
    test_labels = np.where(test_dataset['label'] == 1, 1, -1)

    # Perform ensemble learning analysis
    single_tree_bias, single_tree_variance, bagged_tree_bias, bagged_tree_variance = ensemble_learning(train_features, train_labels, test_features, test_labels)

    # Perform random forest analysis
    rf_single_tree_bias, rf_single_tree_variance, forest_bias, forest_variance = random_forest_learning(train_features, train_labels, test_features, test_labels)

    # Display results
    print(f"Single Tree - Bias^2: {single_tree_bias}, Variance: {single_tree_variance}, Total Error: {single_tree_bias + single_tree_variance}")
    print(f"Bagged Trees - Bias^2: {bagged_tree_bias}, Variance: {bagged_tree_variance}, Total Error: {bagged_tree_bias + bagged_tree_variance}")
    print(f"Single Random Tree - Bias^2: {rf_single_tree_bias}, Variance: {rf_single_tree_variance}, Total Error: {rf_single_tree_bias + rf_single_tree_variance}")
    print(f"Random Forest - Bias^2: {forest_bias}, Variance: {forest_variance}, Total Error: {forest_bias + forest_variance}")

# Entry point of the script
if __name__ == "__main__":
    execute_analysis()

Single Tree - Bias^2: 0.341472, Variance: 0.303968, Total Error: 0.64544
Bagged Trees - Bias^2: 0.30278796, Variance: 0.01485884, Total Error: 0.3176468
Single Random Tree - Bias^2: 0.343608, Variance: 0.30431199999999997, Total Error: 0.64792
Random Forest - Bias^2: 0.30346629999999997, Variance: 0.0164491, Total Error: 0.31991539999999996
